1. devices put in a 20*20 grid area
2. devices collect (device_id, walker_id, time, distance between walker and device) info
    - walker_id is updated every alternative time interval

In [1]:
%load_ext autoreload
%autoreload 2
import sqlite3
import pandas as pd
import collections
import networkx as nx
utils = __import__('04.utils', fromlist=['object'])

In [2]:
cnx = sqlite3.connect('../corona-sniffer/backend/data/database_3000.db')

df_walks = pd.read_sql_query("SELECT * FROM walks", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_contacts = pd.read_sql_query("SELECT * FROM contacts", cnx)

In [3]:
df_contacts.head()

,rolling_id,agent_id,walker_id,time,json,resolved_id
0,81228a81594df2e2fc46,0cb6755f710700140b8e*x-30*y-390,86c7d2ad3d4158cfc115,0,"{""distance"":1.9871228057338732,""agentPos"":{""x""...",None
1,81228a81594df2e2fc46,43e17e257f102e7c5227*x-44.081632653061206*y-39...,86c7d2ad3d4158cfc115,0,"{""distance"":14.216784641469633,""agentPos"":{""x""...",None
2,81228a81594df2e2fc46,c4a74875ab2ae7deab2e*x-30*y-420,86c7d2ad3d4158cfc115,0,"{""distance"":28.161787703261563,""agentPos"":{""x""...",None
3,81228a81594df2e2fc46,086f080c46c32653d3b4*x-14.693877551020364*y-39...,86c7d2ad3d4158cfc115,0,"{""distance"":16.765382255724372,""agentPos"":{""x""...",None
4,81228a81594df2e2fc46,a791163eb39d2c6aa6d4*x-14.693877551020364*y-36...,86c7d2ad3d4158cfc115,0,"{""distance"":29.28281059665801,""agentPos"":{""x"":...",None


In [4]:
walker_ids = df_walkers['id'].tolist()
walker_vs = {}

for walker_id in walker_ids:
    v = utils.get_velocity_of_walker_id(walker_id, df_contacts)
    if v == {}:
        continue
    walker_vs[walker_id] = v

ZeroDivisionError: float division by zero

### Statistical

In [90]:
cnx = sqlite3.connect('../corona-sniffer/backend/data/database_3000.db')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_contacts = pd.read_sql_query("SELECT * FROM contacts", cnx)
df_agents = pd.read_sql_query("SELECT * FROM agents", cnx)

In [91]:
id_to_pos, pos_to_id = utils.get_position(df_agents)

In [98]:
prob_agent_id = utils.get_agent_next_prob(df_walkers, df_contacts)

In [99]:
prob_pos = utils.map_prob_to_pos(prob_agent_id, id_to_pos)

In [100]:
cnx = sqlite3.connect('../corona-sniffer/backend/data/database_3000_test.db')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_contacts = pd.read_sql_query("SELECT * FROM contacts", cnx)
df_agents = pd.read_sql_query("SELECT * FROM agents", cnx)

In [101]:
"""
Get counter of agent_id Key -> agent_id Values
"""
id_to_pos, pos_to_id = utils.get_position(df_agents)
prob_agent_id = utils.map_prob_to_agent_id(prob_pos, pos_to_id)

In [102]:
"""
# agent_id -> next_agent_id
sparse data
"""
print(collections.Counter([i for dic in prob_agent_id.values() for _, i in dic.items()]))

Counter({1: 718, 2: 22, 3: 4, 5: 1, 7: 1, 4: 1})


In [103]:
"""
Map agent id to its most possible next agent.
"""
link_list = {}
for _, row in df_contacts.iterrows():
    agent_id = row['agent_id']
    if agent_id in prob_agent_id and prob_agent_id[agent_id]:
        link_list[agent_id] = max(prob_agent_id[agent_id].items(), key=lambda x: x[1])[0]

In [104]:
real_id_to_agent_dict = collections.defaultdict(set)
idx = 0
for k, v in link_list.items():
    real_id_to_agent_dict[idx].add(k)
    real_id_to_agent_dict[idx].add(v)
    link_list[k] = None
    while v in link_list and link_list[v]:
        k = v
        v = link_list[k]
        real_id_to_agent_dict[idx].add(k)
        real_id_to_agent_dict[idx].add(v)
        link_list[k] = None

In [105]:
agent_id_to_real_dict = {}
for k, v in real_id_to_agent_dict.items():
    for id in v:
        agent_id_to_real_dict[id] = k

In [106]:
walker_id_link_list = {}
for _, row in df_walks_connected.iterrows():
    walker_id = row['walker_id']
    df = df_contacts.loc[df_contacts['walker_id'] == walker_id].sort_values('time')
    for _, row2 in df.iterrows():
        agent_id, time = row2['agent_id'], row2['time']
        
        if agent_id not in prob_agent_id:
            continue
        next_agent_id = prob_agent_id[agent_id]
        for next_id in next_agent_id:
            next_walker_id = df_contacts.loc[(df_contacts['agent_id'] == next_id) & 
                                        (df_contacts['time'] > time)]['walker_id'].tolist()
            if len(next_walker_id) > 0:
                walker_id_link_list[walker_id] = next_walker_id[0]

In [107]:
walker_id_to_real_dict = {}
walker_id_graph = nx.Graph([(i, j) for i, j in walker_id_link_list.items() if i and j])
walker_id_cc = sorted(nx.connected_components(walker_id_graph), key=len, reverse=True)

In [108]:
df_walks_connected = pd.DataFrame(df_walks)
for i, component in enumerate(walker_id_cc):
    new_id = i
    for c in component:
        df_walks_connected = df_walks_connected.replace(c, new_id)

In [109]:
cnx.execute('DROP TABLE IF EXISTS walks_attached')
df_walks_connected.to_sql('walks_attached', con=cnx)

In [110]:
df_contacts

,rolling_id,agent_id,walker_id,time,json,resolved_id
0,628af9e30467a114bc72,3119edf86d793dbf50c7*x-690*y-420,73b9d4f85d4e4b748bbb,0,"{""distance"":9.87161040351828,""agentPos"":{""x"":-...",None
1,628af9e30467a114bc72,112bc3f9c9c1e31058c6*x-690.6122448979592*y-396...,73b9d4f85d4e4b748bbb,0,"{""distance"":14.740671669024156,""agentPos"":{""x""...",None
2,628af9e30467a114bc72,b8c10634fa2779dd1f2c*x-661.2244897959183*y-426...,73b9d4f85d4e4b748bbb,0,"{""distance"":29.387681136542046,""agentPos"":{""x""...",None
3,628af9e30467a114bc72,519b4bc29baa099fd524*x-661.2244897959183*y-396...,73b9d4f85d4e4b748bbb,0,"{""distance"":28.791108006711948,""agentPos"":{""x""...",None
4,a34d6ad77f1345c7fa57,24d983e3d313e5bfe6a4*x-690*y-450,73b9d4f85d4e4b748bbb,1,"{""distance"":25.129946815840036,""agentPos"":{""x""...",None
...,...,...,...,...,...,...
7303,453152cfa6874b544289,cff29d55017bb4b3425b*x720*y308.57142857142867,e4e9953fc7ffe9f012a5,3,"{""distance"":25.773307334378753,""agentPos"":{""x""...",None
7304,56f5a48b4d96a8d6c471,73080cf37c8e3d5a191c*x750*y270,6d2c83d16e04bd9b9112,4,"{""distance"":23.634425855755424,""agentPos"":{""x""...",None
7305,6265743ebeb60060669c,be6272f5f5e3a58494cd*x249.79591836734699*y-602...,a30225e63fe3eb6852df,58,"{""distance"":14.849080826258705,""agentPos"":{""x""...",None
7306,1cced8dcf77d186c53f3,2a345928856c02059fd3*x360*y150,23602e6a4c9b4f9272bf,32,"{""distance"":12.72825441579665,""agentPos"":{""x"":...",None
